In [21]:
import os
os.chdir('/home2/cslab/Music/Mtech-projects/Deploying-Customer-Churn-model/notebook')


In [22]:
%pwd

'/home2/cslab/Music/Mtech-projects/Deploying-Customer-Churn-model/notebook'

In [23]:
os.chdir("../")

In [24]:
%pwd

'/home2/cslab/Music/Mtech-projects/Deploying-Customer-Churn-model'

In [25]:
from dataclasses import dataclass

from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [26]:
from mlFlowProject.constants import *
from mlFlowProject.utils.common import read_yaml, create_directories

In [27]:


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config



In [28]:
import os
import urllib.request as request
import zipfile
from mlFlowProject  import logger
from mlFlowProject.utils.common import get_size

In [29]:
import os
import zipfile
from urllib import request
import logging
from pathlib import Path

logger = logging.getLogger(__name__)

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! With the following info: \n{headers}")
        else:
            file_size = os.path.getsize(self.config.local_data_file)
            logger.info(f"File already exists of size: {file_size} bytes")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extracted files to {unzip_path}")


In [30]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-05 22:30:09,822: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-02-05 22:30:09,823: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-05 22:30:09,825: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-05 22:30:09,826: INFO: common: created directory at: artifacts]
[2025-02-05 22:30:09,827: INFO: common: created directory at: artifacts/data_ingestion]
[2025-02-05 22:30:10,899: INFO: 4249035185: artifacts/data_ingestion/data.zip downloaded! With the following info: 
Server: GitHub.com
Date: Wed, 05 Feb 2025 17:00:10 GMT
Content-Type: text/html; charset=utf-8
Vary: X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, Accept-Encoding, Accept, X-Requested-With
ETag: W/"53b4768b246a512e9bdf04e2d9e3fcbc"
Cache-Control: max-age=0, private, must-revalidate
Strict-Transport-Security: max-age=31536000; includeSubdomains; preload
X-Frame-Options: deny
X-Content-Type-Options: nosniff
X-XSS-Protection: 0
Referrer-Policy: no-referrer-w

BadZipFile: File is not a zip file